In [1]:
COMPONENT='argowf'
FOSSA_DEPENDENCIES = f"../dependencies.{COMPONENT}.yaml"
OUTPUT_FILE = f"../foss.usage.{COMPONENT}.yaml"

In [2]:
print (f"FOSSA_DEPENDENCIES: {FOSSA_DEPENDENCIES}")
print (f"OUTPUT_FILE: {OUTPUT_FILE}")


FOSSA_DEPENDENCIES: ../dependencies.argowf.yaml
OUTPUT_FILE: ../foss.usage.argowf.yaml


In [3]:
import yaml 

with open("obligations_text.yaml") as f:
    license_dict = yaml.load(f, Loader=yaml.FullLoader)

license_dict


{'ISC': '"Including the full license text in a prominent place with the software when the FOSS is distributed."\n',
 'MIT': '"Including the full license text in a prominent place with the software when the FOSS is distributed.\nThis fulfills, The MIT copyright notice and the MIT permission notices shall be included in all copies or\nsubstantial portions of the software (in whatever format, either in binary or in source)"\n',
 'BSD-3-Clause': '"Including the full license text in a prominent place with the software when the FOSS is distributed. \nThis fulfills the following: Redistributions of source code must retain the above copyright notice, this list of conditions and the license disclaimer.\nRedistributions in binary form must reproduce the copyright notice, this list of conditions and the disclaimer in the documentation and/or\nother materials provided with the distribution"\n',
 'Apache-2.0': '"Including the full license text in a prominent place with the software when the FOSS is

In [4]:
! pip install pandas pyyaml jinja2

In [5]:
import yaml, pandas

/var/folders/2y/nm3c9qfx2f18k99cjg94b_1h0000gp/T/ipykernel_28623/1491050532.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import yaml, pandas


In [6]:
with open(FOSSA_DEPENDENCIES, "r") as f:
    dep_dict = yaml.load(f, Loader=yaml.FullLoader)

In [7]:
flat_dict = {}
invalid_entries = []
for entry in dep_dict['dependencies']:
    if entry['bazaar']['register'] == "no":
        primID=entry['bazaar']['prim']
        foss_product_number=primID.split("/")[1]
        nested_dict = {}
        nested_dict["ID"] = entry["ID"]
        usage = entry['bazaar']['component_highlevel_description']
        if len(usage.rstrip('"').lstrip('"').strip()) == 0:
            usage = entry["additional_info"]["fossa-attribution"]["Description"]
        nested_dict["usage"] = usage
        nested_dict["obligation"] = "unknown"
        nested_dict["license"] = "unknown"
        nested_dict["primID"] = primID
        if 'selected_licenses' in entry and len(entry['selected_licenses']) > 0:
            nested_dict["license"] = entry['selected_licenses'][0]
        elif 'selected_licenses' in entry['mimer'] and len(entry['mimer']['selected_licenses']) > 0:
            nested_dict["license"] = entry['mimer']['selected_licenses'][0]
        elif 'licenses' in entry and len(entry['licenses']) >= 1:
            nested_dict["license"] = entry['licenses'][0]
        
        if nested_dict["license"] in license_dict.keys():
            nested_dict["obligation"] = license_dict[nested_dict["license"]]
        elif nested_dict["license"] == "MIT License":
            nested_dict["license"] = "MIT"
            nested_dict["obligation"] = license_dict["MIT"]
        
        if nested_dict["obligation"] == "unknown" or nested_dict["license"] == "unknown":
            invalid_entries.append(nested_dict)
        else:
            if foss_product_number in flat_dict.keys():
                flat_dict[foss_product_number].append(nested_dict)
            else:
                flat_dict[foss_product_number] = [nested_dict]


In [8]:

invalid_entries      

if len(invalid_entries) > 0:
    print("Invalid entries found")
    
    import pandas as pd 
    df = pd.DataFrame(invalid_entries)
    df[["ID", "primID", "license"]].to_csv(f"unknown_licenses_{COMPONENT}.csv", index=False)

else:
    print("No invalid entries found")

No invalid entries found


In [9]:
if len(invalid_entries) > 0:
    df.groupby("license")[["ID"]].count()



In [10]:
from collections import OrderedDict
ordered_dict=OrderedDict(sorted(flat_dict.items()))

In [11]:
def is_ascii(usage_text):
    return all(ord(c) < 128 for c in usage_text)
    

In [12]:
records=[]
nonAsciiUsage = []
quotes='"'
for key,value in ordered_dict.items():
    first_entry = value[0] 
    ordered_dict = OrderedDict()
    ordered_dict["product_number"] = key
    ordered_dict["license"] = first_entry["license"]
    ordered_dict["obligation"] = first_entry["obligation"].rstrip('\n')
    usage = first_entry["usage"].strip('\"')
    ordered_dict["usage"] = f"{quotes}{usage}{quotes}"
    if not is_ascii(ordered_dict["usage"]):
        nonAsciiUsage.append(f"Non-ascii character found in usage for product number {key}", flush=True)
    records.append(ordered_dict)

records[0:10]

[OrderedDict([('product_number', 'CAX1057062'),
              ('license', 'BSD-3-Clause'),
              ('obligation',
               '"Including the full license text in a prominent place with the software when the FOSS is distributed. \nThis fulfills the following: Redistributions of source code must retain the above copyright notice, this list of conditions and the license disclaimer.\nRedistributions in binary form must reproduce the copyright notice, this list of conditions and the disclaimer in the documentation and/or\nother materials provided with the distribution"'),
              ('usage', '"Go OAuth2"')]),
 OrderedDict([('product_number', 'CAX1058617'),
              ('license', 'Apache-2.0'),
              ('obligation',
               '"Including the full license text in a prominent place with the software when the FOSS is distributed.\nThis fulfills the following: Ericsson must give any recipients of the software a copy of the Apache License.\nEricsson must cause any mod

In [13]:
for record in nonAsciiUsage:
    print(record)

In [14]:
from jinja2 import Template

template = Template("""
#
# COPYRIGHT Ericsson 2022
#
#
#
# The copyright to the computer program(s) herein is the property of
#
# Ericsson Inc. The programs may be used and/or copied only with written
#
# permission from Ericsson Inc. or in accordance with the terms and
#
# conditions stipulated in the agreement/contract under which the
#
# program(s) have been supplied.
#

modelVersion: 2.0
fosses:
{%- filter indent(2) -%}
{% for record in records %}
- product_number: {{ record.product_number }}
  license: {{ record.license }}
  obligation: |
    {{ record.obligation | indent(width=4) }}
  usage: |
    {{ record.usage | indent(width=4) -}}
{% endfor %}
{%- endfilter -%}
""")


In [15]:
output = template.render(records=records)

In [16]:
with open(OUTPUT_FILE, 'w') as f:
    f.write(output)